In [41]:
import pandas as pd
import rasterio
import scipy
import numpy as np
from scipy import ndimage
import glob

In [42]:
# file set up

from lab5functions import slopeAspect, reclassAspect, reclassByHisto

dem = rasterio.open(r"C:\Users\catan\Desktop\GEOG5092\lab5\data\data\bigElk_dem.tif")
dem_read = dem.read(1)
slope, aspect = slopeAspect(dem_read, 30)

dem_aspect = reclassAspect(aspect)
dem_slope = reclassByHisto(slope, 10)

landsat_path = glob.glob(r"C:\Users\catan\Desktop\GEOG5092\lab5\data\data\L5_big_elk\*.tif")

with rasterio.open(r"C:\Users\catan\Desktop\GEOG5092\lab5\data\data\fire_perimeter.tif") as fire_file:
    fire_perimeter = fire_file.read()

In [43]:
# part 1

band3 = []
band4 = []
NDVIarr = []
final_printlist = []
flat_rr = []
year = []


for image in landsat_path:
    if 'B3.tif' in image:
        band3.append(image)
    if 'B4.tif' in image:
        band4.append(image)
        
healthy_area = np.where(fire_perimeter == 2)
burned_area = np.where(fire_perimeter == 1)
    
for b3, b4 in zip(band3, band4):
    year.append(b3[-11:-7])
    with rasterio.open(b3, 'r') as data:
        band3_arr = data.read()
    with rasterio.open(b4, 'r') as beta:
        band4_arr = beta.read()
    NDVI = (band4_arr - band3_arr) / (band4_arr + band3_arr)
    healthy_area = np.where(fire_perimeter == 2)
    burned_area = np.where(fire_perimeter == 1)
    ndviMean = NDVI[healthy_area].mean()
    rr = NDVI / ndviMean
    burnt_mean = rr[burned_area].mean()
    final_printlist.append(burnt_mean)
    
    flat = rr.flatten()
    flat_rr.append(flat)
    
stacked = np.vstack(flat_rr)
coeff_recov = np.polyfit(range(10), stacked, 1)[0]
final_recov = coeff_recov.reshape(280,459)
mean_coef = np.where(fire_perimeter == 1, final_recov, np.nan)
rrMean_coeff = np.nanmean(mean_coef)

for yr,rat in zip(year,final_printlist):
    print("In ", yr, "the mean recovery ratio was", rat)
print("The mean coefficient of recovery is", rrMean_coeff)

In  2002 the mean recovery ratio was 0.41126525
In  2003 the mean recovery ratio was 0.5412664
In  2004 the mean recovery ratio was 0.51346225
In  2005 the mean recovery ratio was 0.61524826
In  2006 the mean recovery ratio was 0.7161738
In  2007 the mean recovery ratio was 0.70540994
In  2008 the mean recovery ratio was 0.739514
In  2009 the mean recovery ratio was 0.7126317
In  2010 the mean recovery ratio was 0.58509773
In  2011 the mean recovery ratio was 0.6258852
The mean coefficient of recovery is 0.021795632717297512


In [44]:
# zonal statistics function

def zonal_stats_table(zone_raster, value_raster, csv_output):
          
    min_list = []
    max_list = []
    mean_list = []
    std_list = []
    count_list = []
    zone = []
    
    for u in np.unique(zone_raster):
        ras = np.where(zone_raster == u, u, np.nan)
        min_list.append(np.nanmin(ras * value_raster))
        max_list.append(np.nanmax(ras * value_raster))
        mean_list.append(np.nanmean(ras * value_raster))
        std_list.append(np.nanstd(ras * value_raster))
        count_list.append(np.where(zone_raster == u, 1, 0).sum())
        zone.append(int(u))
    
    stats = {'zone': zone, 'min': min_list, 'max': max_list, 'mean': mean_list,
             'std': std_list, 'count': count_list}
    df = pd.DataFrame(stats)
    df.to_csv(csv_output)
    return df

In [45]:
zonal_stats_table(dem_slope, final_recov, "slope.csv")

,zone,min,max,mean,std,count
0,1,-0.165033,0.118608,0.011305,0.026438,9961
1,2,-0.350247,0.233357,0.004480,0.048113,23490
2,3,-0.528085,0.314058,0.002555,0.065139,26247
3,4,-0.383108,0.424008,0.008198,0.078669,24596
4,5,-0.475438,0.551415,0.014652,0.090263,20579
5,6,-0.524314,0.723230,0.032466,0.114893,14060
6,7,-0.419862,0.799784,0.064447,0.156556,6938
7,8,-0.366635,0.914364,0.097692,0.195011,2222
8,9,-0.312660,1.060375,0.159624,0.225148,392
9,10,-0.203882,0.671084,0.140781,0.167236,35


In [46]:
zonal_stats_table(dem_aspect, final_recov, "aspect.csv")

,zone,min,max,mean,std,count
0,1,-0.128122,0.120538,0.008561,0.023988,14334
1,2,-0.251284,0.219043,0.002318,0.045304,20156
2,3,-0.495100,0.350035,-0.003828,0.068812,26609
3,4,-0.409488,0.377837,0.006006,0.076452,18195
4,5,-0.398906,0.474215,0.015430,0.085477,14202
5,6,-0.479095,0.583648,0.029805,0.111335,14232
6,7,-1.232199,0.686659,0.054114,0.142031,10763
7,8,-0.760471,0.826363,0.108217,0.190029,10029


In [47]:
with rasterio.open(f'Coeff_recov.tif', 'w',
                 driver = 'GTiff',
                 height= final_recov.shape[0],
                 width=final_recov.shape[1],
                 count=1,
                 dtype=np.float64,
                 crs=data.crs,
                 transform=data.transform,
                 ndata =data.nodata
                 ) as tif_data:
        tif_data.write(final_recov,1)

In [48]:
print("The mean recovery coeffecient for the slope is higher meaning that the vegetation has recovered faster than the aspect which has a lower mean recovery coeffecient. Elevation did not impact recovery of the vegetation as much as direction of the slope.")

The mean recovery coeffecient for the slope is higher meaning that the vegetation has recovered faster than the aspect which has a lower mean recovery coeffecient. Elevation did not impact recovery of the vegetation as much as direction of the slope.
